In [1]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
import json
import torch
from datasets import load_dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Carregar o tokenizer e o modelo
model_name = 'microsoft/phi-1_5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
# Carregar os dados do arquivo JSON
with open('seus_dados.json', 'r') as f:
    data = json.load(f)

# Estrutura do dataset dos dados no formato Alpaca (ou similar)
# O formato deve conter campos como 'prompt' e 'completion'
train_data = [
    {'prompt': item['prompt'], 'completion': item['completion']} 
    for item in data
]

# Criar um dataset para o treinamento
from datasets import Dataset
train_dataset = Dataset.from_list(train_data)

In [4]:
# Tokenize os dados
def tokenize_function(examples):
    return tokenizer(examples['prompt'], truncation=True,
                     padding='max_length', max_length=512)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

In [5]:
# Configuração do treinamento
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    eval_strategy='steps',
)

# Treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    # eval_dataset=val_dataset  # Você pode adicionar um conjunto de validação aqui
)

In [6]:
# Iniciar o treinamento
trainer.train()

In [7]:
# Salvar o modelo finamente ajustado
trainer.save_model('./meu_modelo')
tokenizer.save_pretrained('./meu_modelo')